In [11]:
import numpy as np
import random

grid_size = 4
goal_state = (3, 3)
actions = ["up", "down", "left", "right"]
action_dict = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}

# Initialize Q-table and returns dictionary
Q = np.zeros((grid_size, grid_size, len(actions)))
returns = {}  # Stores cumulative returns for state-action pairs
gamma = 0.9  # Discount factor
alpha = 0.1  # Learning rate
episodes = 10  # Number of episodes

# Reward function
def get_reward(state, next_state):
    if next_state == goal_state:
        return 10
    elif next_state[0] < 0 or next_state[0] >= grid_size or next_state[1] < 0 or next_state[1] >= grid_size:
        return -5  # Hitting a wall
    else:
        return -1  # Regular step cost

# Generate an episode (returns a list of (state, action, reward))
def generate_episode():
    episode = []
    state = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))  # Start from random state
    
    while state != goal_state:
      
        action = random.choice(actions)  # Explore randomly
        next_state = (state[0] + action_dict[action][0], state[1] + action_dict[action][1])
        # print(state,action,next_state)
        # Ensure state is within bounds
        if next_state[0] < 0 or next_state[0] >= grid_size or next_state[1] < 0 or next_state[1] >= grid_size:
            next_state = state  # Stay in place if hitting a wall
        
        reward = get_reward(state, next_state)
        episode.append((state, action, reward))
        state = next_state  # Move to the next state
    
    return episode

# Monte Carlo learning
for episode_num in range(episodes):
    print('episode 111111')
    episode = generate_episode()
    # print(len(episode),episode)
    G = 0  # Initialize return
    visited = set()  # Track first visits

    for t in reversed(range(len(episode))):  # Work backwards from end to start
        state, action, reward = episode[t]
        G = reward + gamma * G  # Compute return

        if (state, action) not in visited:  # First-visit MC
            visited.add((state, action))
            i, j = state
            a_index = actions.index(action)
            
            # Update returns dictionary
            if (i, j, a_index) not in returns:
                returns[(i, j, a_index)] = []
            returns[(i, j, a_index)].append(G)
            # print(returns)
            # Update Q-value with the mean return
            Q[i, j, a_index] = np.mean(returns[(i, j, a_index)])
    print(Q)
# Derive policy from Q-values
policy = np.chararray((grid_size, grid_size), unicode=True)
for i in range(grid_size):
    for j in range(grid_size):
        if (i, j) == goal_state:
            policy[i, j] = "G"
        else:
            best_action = np.argmax(Q[i, j])
            print(np.argmax(Q[i,j]),Q[i,j])
            policy[i, j] = actions[best_action][0].upper()  # First letter of the best action

print("Learned Policy:")
print(policy)


episode 111111
[[[ 0.          0.          0.         -3.0264312 ]
  [ 0.          0.          0.         -2.25159022]
  [ 0.62882     1.8098      0.         -7.29829656]
  [-6.99810729 -6.66456366 -7.56846691  0.        ]]

 [[-3.72378808 -9.05797426  0.          0.        ]
  [-8.40467114  4.58       -4.35140927 -9.44374322]
  [ 0.          0.          3.122       0.        ]
  [ 0.          0.         -5.42464151 -5.88217736]]

 [[ 0.          0.          0.         -8.95330473]
  [-8.56420402  6.2         0.         -8.83700526]
  [ 0.          0.         -8.70778362  0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.          0.         -9.70438234 -9.67153593]
  [ 0.          0.          0.          8.        ]
  [-9.59448881  0.          0.         10.        ]
  [ 0.          0.          0.          0.        ]]]
episode 111111
[[[ 0.          0.          0.         -3.0264312 ]
  [ 0.          0.          0.         -1.34282611]
  [-0.81138511  1.21931   

In [ ]:
import numpy as np
import random

# Define the Gridworld size
grid_size = 4
goal_state = (3, 3)
actions = ["up", "down", "left", "right"]
action_dict = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}

# Initialize Q-table with zeros
Q = np.zeros((grid_size, grid_size, len(actions)))

# Hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration probability
episodes = 10000  # Number of episodes

# Reward function
def get_reward(state, next_state):
    if next_state == goal_state:
        return 10
    elif next_state[0] < 0 or next_state[0] >= grid_size or next_state[1] < 0 or next_state[1] >= grid_size:
        return -5  # Hitting a wall
    else:
        return -1  # Regular step cost

# Q-learning algorithm
for episode in range(episodes):
    state = (0, 0)  # Start from top-left corner
    
    while state != goal_state:
        # Choose action using epsilon-greedy
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)  # Exploration
        else:
            action = actions[np.argmax(Q[state[0], state[1]])]  # Exploitation

        # Perform action
        next_state = (state[0] + action_dict[action][0], state[1] + action_dict[action][1])

        # Stay in place if hitting a wall
        if next_state[0] < 0 or next_state[0] >= grid_size or next_state[1] < 0 or next_state[1] >= grid_size:
            next_state = state

        # Get reward
        reward = get_reward(state, next_state)

        # Q-learning update
        Q[state[0], state[1], actions.index(action)] += alpha * (
            reward + gamma * np.max(Q[next_state[0], next_state[1]]) - Q[state[0], state[1], actions.index(action)]
        )

        state = next_state  # Move to the next state

# Extract learned policy
policy = np.chararray((grid_size, grid_size), unicode=True)
for i in range(grid_size):
    for j in range(grid_size):
        if (i, j) == goal_state:
            policy[i, j] = "G"
        else:
            best_action = np.argmax(Q[i, j])
            policy[i, j] = actions[best_action][0].upper()  # First letter of best action

# Print the learned policy
print("Learned Policy:")
print(policy)


Learned Policy:
[['D' 'R' 'D' 'D']
 ['R' 'R' 'R' 'D']
 ['R' 'R' 'R' 'D']
 ['R' 'R' 'R' 'G']]


Q-learning vs. Monte Carlo


Feature	Q-learning	Monte Carlo

Update Timing	After each step	After each episode

Exploration	Epsilon-greedy	Starts randomly

Continuous Tasks	Yes	No

Convergence Speed	Faster	Slower